In [2]:
import os
import pandas as pd
import numpy as np
from tensorflow.keras.utils import to_categorical

In [3]:
WINDOW_SIZE = 25
NUMBER_OF_FEATURES = 4
NUMBER_OF_CLASSES = 2

MOTION_CLASSES = ['STAND', 'STEPS']
MOTIONTYPE_CLASSES = ['SML', 'MED', 'LAR']
MOTIONSPEED_CLASSES = ['SLOW', 'AVERAGE', 'FAST']

# Get the current working directory
CURRENT_DIRECTORY = os.getcwd()
# Get the parent directory
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)
PARENT_DIRECTORY = os.path.dirname(CURRENT_DIRECTORY)

DATA_FOLDER = PARENT_DIRECTORY + '\\processed-training-data\\4-PROCESSED-DATA\TEST2\\'

# numpy data folder
NUMPY_DATA_FOLDER_FILE_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\'

# v0 data (not normalized, use to generate more data)
ALL_X_TEST_CLASSIFY_MOTION_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-X-TEST-CLASSIFY-MOTION-V0.npy'
ALL_Y_TEST_CLASSIFY_MOTION_PATH = PARENT_DIRECTORY + '\\NeuralNetwork\\np-saved-data\\test\\ALL-Y-TEST-CLASSIFY-MOTION-V0.npy'

In [49]:
LABEL_TO_CATEGORY_MOTION = {label: category for category, label in enumerate(MOTION_CLASSES)}
LABEL_TO_CATEGORY_MOTIONTYPE = {label: category for category, label in enumerate(MOTIONTYPE_CLASSES)}
LABEL_TO_CATEGORY_MOTIONSPEED = {label: category for category, label in enumerate(MOTIONSPEED_CLASSES)}

print(LABEL_TO_CATEGORY_MOTION)
print(LABEL_TO_CATEGORY_MOTIONTYPE)
print(LABEL_TO_CATEGORY_MOTIONSPEED)

{'STAND': 0, 'STEPS': 1}
{'SML': 0, 'MED': 1, 'LAR': 2}
{'SLOW': 0, 'AVERAGE': 1, 'FAST': 2}


In [50]:
file_names = [file for file in os.listdir(DATA_FOLDER) if file.endswith('.xlsx') and os.path.isfile(os.path.join(DATA_FOLDER, file))]

print(file_names)

['PROC-TEST2-STEPS-LR-LAR-130BPM.xlsx', 'PROC-TEST2-STEPS-LR-LAR-132BPM.xlsx', 'PROC-TEST2-STEPS-LR-LAR-180BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-184BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-196BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-260BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-264BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-45BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-46BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-49BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-65BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-66BPM-AUGMENT.xlsx', 'PROC-TEST2-STEPS-LR-LAR-90BPM.xlsx', 'PROC-TEST2-STEPS-LR-LAR-92BPM.xlsx', 'PROC-TEST2-STEPS-LR-LAR-98BPM.xlsx']


In [51]:
def normalizeSensorData(sensor_input):
    return (sensor_input + 180)/360


In [52]:
def df2Xy(df, windowSize=5):
  X = []
  y_Motion= []

  # NORMALIZE THE SENSOR DATA
  #df["L_Pitch_Delta"] = df["L_Pitch_Delta"].apply(normalizeSensorData)
  #df["L_Roll_Delta"] = df["L_Roll_Delta"].apply(normalizeSensorData)
  #df["R_Pitch_Delta"] = df["R_Pitch_Delta"].apply(normalizeSensorData)
  #df["R_Roll_Delta"] = df["R_Roll_Delta"].apply(normalizeSensorData)

  for i in range( len(df) - windowSize + 1):
    # inputs: X rows
    # form a new input which has size of our windowSize
    input_data_list = []

    # loop through each row in our windowsize
    for j in range(windowSize):
        # fetch sensor data for this row
        row_values = df.loc[i + j, ['L_Pitch_Delta', 'L_Roll_Delta', 'R_Pitch_Delta', 'R_Roll_Delta']].values.tolist()

        # add row values to the input
        input_data_list.append(row_values)

    # turnn list into array to do arthimetic
    input_data_array = np.array(input_data_list)

    # remove all negative sinces we want to the total change (we dont care which direction)
    input_data_array = np.abs(input_data_array)

    # calculate the total change for each sennsor value (TOTAL POSITIVE SENSOR CHANGES)
    input_data_array = np.sum(input_data_array, axis=0)

    if(np.isnan(input_data_array).any() == False):
      # turn back to list
      input_data_list = input_data_array.tolist()

      # add our input to our total inputs, marked as X
      X.append(input_data_list)

      # outputs: y labels
      label = df.loc[i + (windowSize - 1), ['Class_Motion']].values.tolist()

      y_Motion.append(label)

  return (np.array(X), np.array(y_Motion))

In [53]:
ALL_X_MOTION = np.empty((0, NUMBER_OF_FEATURES))  # List to store all X training data
ALL_Y_MOTION = np.empty((0))  # List to store all Y training data

In [54]:
def processData(ALL_X_MOTION, ALL_Y_MOTION):
    for fileName in file_names:
        # Read the Excel file
        df = pd.read_excel(DATA_FOLDER + fileName)

        xTest, yTest_Motion = df2Xy(df, WINDOW_SIZE)

        yTest_Motion = yTest_Motion.reshape(-1)
        
        ALL_X_MOTION = np.concatenate([ALL_X_MOTION, xTest])
        ALL_Y_MOTION = np.concatenate([ALL_Y_MOTION, yTest_Motion])
        
        print('xTest.shape:', xTest.shape)
        print('ALL_X.shape:', ALL_X_MOTION.shape, ' ALL_Y_MOTION.shape:', ALL_Y_MOTION.shape)
        print('-------------------------------')

    # Create the folder if it doesn't exist
    if not os.path.exists(NUMPY_DATA_FOLDER_FILE_PATH):
        os.makedirs(NUMPY_DATA_FOLDER_FILE_PATH)

    # convert to numerical labels (originally text labels)
    numerical_motion_label = np.vectorize(LABEL_TO_CATEGORY_MOTION.get)(ALL_Y_MOTION)
    

    # Assuming your numpy array is called 'data_array'
    np.save(ALL_X_TEST_CLASSIFY_MOTION_PATH, ALL_X_MOTION)
    np.save(ALL_Y_TEST_CLASSIFY_MOTION_PATH, numerical_motion_label)


In [55]:
if not os.path.exists(ALL_X_TEST_CLASSIFY_MOTION_PATH):
    processData(ALL_X_MOTION, ALL_Y_MOTION)

ALL_X_TEST_CLASSIFY_MOTION = np.load(ALL_X_TEST_CLASSIFY_MOTION_PATH)
ALL_Y_TEST_CLASSIFY_MOTION = np.load(ALL_Y_TEST_CLASSIFY_MOTION_PATH)

print(ALL_X_TEST_CLASSIFY_MOTION.shape)
print(ALL_Y_TEST_CLASSIFY_MOTION.shape)

xTest.shape: (3771, 4)
ALL_X.shape: (3771, 4)  ALL_Y_MOTION.shape: (3771,)
-------------------------------
xTest.shape: (3618, 4)
ALL_X.shape: (7389, 4)  ALL_Y_MOTION.shape: (7389,)
-------------------------------
xTest.shape: (3581, 4)
ALL_X.shape: (10970, 4)  ALL_Y_MOTION.shape: (10970,)
-------------------------------
xTest.shape: (3487, 4)
ALL_X.shape: (14457, 4)  ALL_Y_MOTION.shape: (14457,)
-------------------------------
xTest.shape: (3500, 4)
ALL_X.shape: (17957, 4)  ALL_Y_MOTION.shape: (17957,)
-------------------------------
xTest.shape: (3770, 4)
ALL_X.shape: (21727, 4)  ALL_Y_MOTION.shape: (21727,)
-------------------------------
xTest.shape: (3617, 4)
ALL_X.shape: (25344, 4)  ALL_Y_MOTION.shape: (25344,)
-------------------------------
xTest.shape: (7182, 4)
ALL_X.shape: (32526, 4)  ALL_Y_MOTION.shape: (32526,)
-------------------------------
xTest.shape: (6994, 4)
ALL_X.shape: (39520, 4)  ALL_Y_MOTION.shape: (39520,)
-------------------------------
xTest.shape: (7020, 4)


In [56]:
print(np.max(ALL_X_TEST_CLASSIFY_MOTION))

180.0
